In [3]:
import os, mlflow
from dotenv import load_dotenv

load_dotenv(override=True)
EXPERIMENT_NAME = "/Users/roiflores.2213@gmail.com/nyc-taxi-experiments"

mlflow.set_tracking_uri("databricks")
experiment = mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

In [4]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer

In [5]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [6]:
df_train = read_dataframe('../data/green_tripdata_2025-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2025-02.parquet')

In [7]:
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

In [8]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

X_val = preprocess(df_val, dv)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [10]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-02")

In [11]:
import math
import optuna
import pathlib
import xgboost as xgb
from optuna.samplers import TPESampler
from mlflow.models.signature import infer_signature

c:\Users\Roi_f\PCD\nyc-taxi-predictions-2025\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [13]:
def objective(trial: optuna.trial.Trial):
    params = {
        "max_depth": trial.suggest_int("max_depth", 4, 100),
        "learning_rate": trial.suggest_float("learning_rate", math.exp(-3), 1.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha",   math.exp(-5), math.exp(-1), log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", math.exp(-6), math.exp(-1), log=True),
        "min_child_weight": trial.suggest_float("min_child_weight", math.exp(-1), math.exp(3), log=True),
        "objective": "reg:squarederror",  
        "seed": 42, 
    }
    
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "xgboost")
        mlflow.log_params(params)
        
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, "validation")],
            early_stopping_rounds=10
        )
        
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        
        mlflow.log_metric("rmse", rmse)
        
        signature = infer_signature(X_val, y_pred)
        
        mlflow.xgboost.log_model(
            booster,
            name="model",
            input_example=X_val[:5],
            signature=signature
        )
        
    return rmse

In [14]:
mlflow.xgboost.autolog(log_models=False)

# ------------------------------------------------------------
# Crear el estudio de Optuna
#    - Usamos TPE (Tree-structured Parzen Estimator) como sampler.
#    - direction="minimize" porque queremos minimizar el RMSE.
# ------------------------------------------------------------
sampler = TPESampler(seed=42)
study = optuna.create_study(direction="minimize", sampler=sampler)

# ------------------------------------------------------------
# Ejecutar la optimización (n_trials = número de intentos)
#    - Cada trial ejecuta la función objetivo con un set distinto de hiperparámetros.
#    - Abrimos un run "padre" para agrupar toda la búsqueda.
# ------------------------------------------------------------
with mlflow.start_run(run_name="XGBoost Hyperparameter Optimization (Optuna)", nested=True):
    study.optimize(objective, n_trials=3)

    # --------------------------------------------------------
    # Recuperar y registrar los mejores hiperparámetros
    # --------------------------------------------------------
    best_params = study.best_params
    # Asegurar tipos/campos fijos (por claridad y consistencia)
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"

    mlflow.log_params(best_params)

    # Etiquetas del run "padre" (metadatos del experimento)
    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "xgboost",
        "feature_set_version": 1,
    })
    
    mlflow.xgboost.autolog(log_models=False)

# ------------------------------------------------------------
# Crear el estudio de Optuna
#    - Usamos TPE (Tree-structured Parzen Estimator) como sampler.
#    - direction="minimize" porque queremos minimizar el RMSE.
# ------------------------------------------------------------
sampler = TPESampler(seed=42)
study = optuna.create_study(direction="minimize", sampler=sampler)

# ------------------------------------------------------------
# Ejecutar la optimización (n_trials = número de intentos)
#    - Cada trial ejecuta la función objetivo con un set distinto de hiperparámetros.
#    - Abrimos un run "padre" para agrupar toda la búsqueda.
# ------------------------------------------------------------
with mlflow.start_run(run_name="XGBoost Hyperparameter Optimization (Optuna)", nested=True):
    study.optimize(objective, n_trials=3)

    # --------------------------------------------------------
    # Recuperar y registrar los mejores hiperparámetros
    # --------------------------------------------------------
    best_params = study.best_params
    # Asegurar tipos/campos fijos (por claridad y consistencia)
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"

    mlflow.log_params(best_params)

    # Etiquetas del run "padre" (metadatos del experimento)
    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "xgboost",
        "feature_set_version": 1,
    })

    # --------------------------------------------------------
    # 7) Entrenar un modelo FINAL con los mejores hiperparámetros
    #    (normalmente se haría sobre train+val o con CV; aquí mantenemos el patrón original)
    # --------------------------------------------------------
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, "validation")],
        early_stopping_rounds=10,
    )

    # Evaluar y registrar la métrica final en validación
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # --------------------------------------------------------
    # 8) Guardar artefactos adicionales (p. ej. el preprocesador)
    # --------------------------------------------------------
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

    # La "signature" describe la estructura esperada de entrada y salida del modelo:
    # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
    # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
    # Si X_val es la matriz dispersa (scipy.sparse) salida de DictVectorizer:
    feature_names = dv.get_feature_names_out()
    input_example = pd.DataFrame(X_val[:5].toarray(), columns=feature_names)

    # Para que las longitudes coincidan, usa el mismo slice en y_pred
    signature = infer_signature(input_example, y_val[:5])

    # Guardar el modelo del trial como artefacto en MLflow.
    mlflow.xgboost.log_model(
        booster,
        name="model",
        input_example=input_example,
        signature=signature
    )

[I 2025-10-21 21:25:09,992] A new study created in memory with name: no-name-b163cf2c-6819-4826-b20b-55626b2af41c


[0]	validation-rmse:5.98960
[1]	validation-rmse:5.92413
[2]	validation-rmse:5.92287
[3]	validation-rmse:5.92324
[4]	validation-rmse:5.92483
[5]	validation-rmse:5.92489
[6]	validation-rmse:5.92498
[7]	validation-rmse:5.92343
[8]	validation-rmse:5.92347
[9]	validation-rmse:5.92467
[10]	validation-rmse:5.92456
[11]	validation-rmse:5.92460


c:\Users\Roi_f\PCD\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:169: UserWarning: [21:25:22] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/21 21:25:29 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
c:\Users\Roi_f\PCD\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [21:25:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)
2025/10/21 21:25:29 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if

🏃 View run glamorous-mouse-314 at: https://dbc-8e655ea0-6144.cloud.databricks.com/ml/experiments/2695805265605501/runs/93104e236aac4f33a6043f64d4771c81
🧪 View experiment at: https://dbc-8e655ea0-6144.cloud.databricks.com/ml/experiments/2695805265605501
[0]	validation-rmse:8.79510
[1]	validation-rmse:8.50586
[2]	validation-rmse:8.24215
[3]	validation-rmse:8.00493
[4]	validation-rmse:7.78830
[5]	validation-rmse:7.59575
[6]	validation-rmse:7.41983
[7]	validation-rmse:7.26129
[8]	validation-rmse:7.11270
[9]	validation-rmse:6.98583
[10]	validation-rmse:6.87056
[11]	validation-rmse:6.76533
[12]	validation-rmse:6.67029
[13]	validation-rmse:6.58378
[14]	validation-rmse:6.50773
[15]	validation-rmse:6.43768
[16]	validation-rmse:6.37633
[17]	validation-rmse:6.32116
[18]	validation-rmse:6.27142
[19]	validation-rmse:6.22765
[20]	validation-rmse:6.18963
[21]	validation-rmse:6.15802
[22]	validation-rmse:6.12794
[23]	validation-rmse:6.10295
[24]	validation-rmse:6.07940
[25]	validation-rmse:6.05863
[26

c:\Users\Roi_f\PCD\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:169: UserWarning: [21:25:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/21 21:25:47 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
c:\Users\Roi_f\PCD\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [21:25:47] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)
[I 2025-10-21 21:25:52,401] Trial 1 finished with value: 5.902611653559419 and parameters: {'max_depth': 19, 'learning_rate': 0.059264241587996896, 'reg_alpha': 0.215392

...

🏃 View run illustrious-stag-936 at: https://dbc-8e655ea0-6144.cloud.databricks.com/ml/experiments/2695805265605501/runs/288807c416db440db30e1ecc2e6e44e6
🧪 View experiment at: https://dbc-8e655ea0-6144.cloud.databricks.com/ml/experiments/2695805265605501
[0]	validation-rmse:5.90396
[1]	validation-rmse:5.88901
[2]	validation-rmse:5.88875
[3]	validation-rmse:5.88908
[4]	validation-rmse:5.88973
[5]	validation-rmse:5.88241
[6]	validation-rmse:5.88348
[7]	validation-rmse:5.88292
[8]	validation-rmse:5.87483
[9]	validation-rmse:5.87457
[10]	validation-rmse:5.87441
[11]	validation-rmse:5.87453
[12]	validation-rmse:5.87449
[13]	validation-rmse:5.87444
[14]	validation-rmse:5.87358
[15]	validation-rmse:5.87331
[16]	validation-rmse:5.87310
[17]	validation-rmse:5.87302
[18]	validation-rmse:5.87068
[19]	validation-rmse:5.87074
[20]	validation-rmse:5.87061
[21]	validation-rmse:5.87062
[22]	validation-rmse:5.87063
[23]	validation-rmse:5.87068
[24]	validation-rmse:5.87073
[25]	validation-rmse:5.87077
[2

c:\Users\Roi_f\PCD\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:169: UserWarning: [21:26:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/21 21:26:55 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
c:\Users\Roi_f\PCD\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [21:26:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)
[I 2025-10-21 21:26:57,405] Trial 2 finished with value: 5.873624736518469 and parameters: {'max_depth': 5, 'learning_rate': 0.9136840519292247, 'reg_alpha': 0.188203879

🏃 View run thundering-moth-213 at: https://dbc-8e655ea0-6144.cloud.databricks.com/ml/experiments/2695805265605501/runs/cc4c1a78be3847ee9b50d286062c480d
🧪 View experiment at: https://dbc-8e655ea0-6144.cloud.databricks.com/ml/experiments/2695805265605501
[0]	validation-rmse:5.90396
[1]	validation-rmse:5.88901
[2]	validation-rmse:5.88875
[3]	validation-rmse:5.88908
[4]	validation-rmse:5.88973
[5]	validation-rmse:5.88241
[6]	validation-rmse:5.88348
[7]	validation-rmse:5.88292
[8]	validation-rmse:5.87483
[9]	validation-rmse:5.87457
[10]	validation-rmse:5.87441
[11]	validation-rmse:5.87453
[12]	validation-rmse:5.87449
[13]	validation-rmse:5.87444
[14]	validation-rmse:5.87358
[15]	validation-rmse:5.87331
[16]	validation-rmse:5.87310
[17]	validation-rmse:5.87302
[18]	validation-rmse:5.87068
[19]	validation-rmse:5.87074
[20]	validation-rmse:5.87061
[21]	validation-rmse:5.87062
[22]	validation-rmse:5.87063
[23]	validation-rmse:5.87068
[24]	validation-rmse:5.87073
[25]	validation-rmse:5.87077
[26

c:\Users\Roi_f\PCD\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:169: UserWarning: [21:27:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/21 21:27:09 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
c:\Users\Roi_f\PCD\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [21:27:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)


🏃 View run XGBoost Hyperparameter Optimization (Optuna) at: https://dbc-8e655ea0-6144.cloud.databricks.com/ml/experiments/2695805265605501/runs/5b3648df83a54714bba13ed23fcc04fe
🧪 View experiment at: https://dbc-8e655ea0-6144.cloud.databricks.com/ml/experiments/2695805265605501


In [15]:
model_name = "workspace.default.nyc-taxi-model"

In [16]:
run_id = input("Ingrese el run id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name=model_name
)

Successfully registered model 'workspace.default.nyc-taxi-model'.


RestException: RESOURCE_DOES_NOT_EXIST: Run '0' not found.

In [17]:
runs = mlflow.search_runs(
    experiment_names= [EXPERIMENT_NAME],
    order_by=["metrics.rmse ASC"],
    output_format="list"
)

if len(runs) > 0:
    best_run = runs[0]
    print("🏆 Champion Run encontrado:")
    print(f"Run ID: {best_run.info.run_id}")
    print(f"RMSE: {best_run.data.metrics['rmse']}")
    print(f"Params: {best_run.data.params}")
    
else:
    print("⚠️ No se encontraron runs con métrica RMSE.")

🏆 Champion Run encontrado:
Run ID: cc4c1a78be3847ee9b50d286062c480d
RMSE: 5.873624736518469
Params: {'custom_metric': 'None', 'early_stopping_rounds': '10', 'learning_rate': '0.9136840519292247', 'max_depth': '5', 'maximize': 'None', 'min_child_weight': '0.7613210498541186', 'num_boost_round': '100', 'objective': 'reg:squarederror', 'reg_alpha': '0.18820387978911576', 'reg_lambda': '0.007166739666045858', 'seed': '42', 'verbose_eval': 'True'}


In [18]:
result = mlflow.register_model(
    model_uri=f"runs:/{best_run.info.run_id}/model",
    name=model_name
)

Registered model 'workspace.default.nyc-taxi-model' already exists. Creating a new version of this model...
2025/10/21 21:28:21 WARNING mlflow.tracking._model_registry.fluent: Run with id cc4c1a78be3847ee9b50d286062c480d has no artifacts at artifact path 'model', registering model based on models:/m-cc5012ff44294d49bb0b0e8fb540d03e instead
Uploading artifacts: 100%|██████████| 8/8 [00:04<00:00,  1.69it/s]
Created version '1' of model 'workspace.default.nyc-taxi-model'.


In [19]:
from mlflow import MlflowClient

client = MlflowClient()

In [20]:
model_version = result.version
new_alias = "Champion"

client.set_registered_model_alias(
    name=model_name,
    alias=new_alias,
    version=result.version
)

In [21]:
from datetime import datetime

date = datetime.today()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1761103707707, current_stage=None, deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='The model version 1 was transitioned to Champion on 2025-10-21 21:31:03.170779', last_updated_timestamp=1761103865151, metrics=[<Metric: dataset_digest='', dataset_name='', key='best_iteration', model_id='m-cc5012ff44294d49bb0b0e8fb540d03e', run_id='cc4c1a78be3847ee9b50d286062c480d', step=0, timestamp=1761103604604, value=20.0>,
 <Metric: dataset_digest='', dataset_name='', key='rmse', model_id='m-cc5012ff44294d49bb0b0e8fb540d03e', run_id='cc4c1a78be3847ee9b50d286062c480d', step=0, timestamp=1761103607967, value=5.873624736518469>,
 <Metric: dataset_digest='', dataset_name='', key='stopped_iteration', model_id='m-cc5012ff44294d49bb0b0e8fb540d03e', run_id='cc4c1a78be3847ee9b50d286062c480d', s

In [22]:
import mlflow.pyfunc

model_version_uri = f"models:/{model_name}@Champion"

champion_version = mlflow.pyfunc.load_model(model_version_uri)
champion_version.predict(X_val)

c:\Users\Roi_f\PCD\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [21:32:23] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)


array([ 4.7416825, 24.05205  , 29.134676 , ..., 21.56244  , 13.990461 ,
       24.05205  ], shape=(44218,), dtype=float32)